<a href="https://colab.research.google.com/github/jrakhshanda/Text-Mining/blob/main/pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install biopython
!pip install  gatenlp
!pip install scispacy 
!pip install swifter

In [ ]:
#!python -m spacy download en_core_web_sm
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_core_sci_lg-0.2.4.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_ner_bionlp13cg_md-0.2.4.tar.gz

In [5]:
print(scispacy.__version__)

0.3.0


In [28]:
import spacy
import scispacy
import swifter
import pandas as pd
from spacy import displacy


import en_core_sci_lg
import en_ner_bionlp13cg_md

from scispacy.abbreviation import AbbreviationDetector
from scispacy.linking import EntityLinker
from collections import OrderedDict,Counter

from pprint import pprint
from tqdm import tqdm
tqdm.pandas()

from IPython.display import clear_output
clear_output() #for clarity

In [25]:
#general importation
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

import time
import random
import matplotlib.pyplot as plt
%matplotlib inline

#Natural Language processing
import regex as re
import nltk
from nltk.stem.porter import PorterStemmer 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import linear_model, svm
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, accuracy_score, recall_score

import gatenlp

import gensim
from gensim.models.word2vec import Word2Vec

from google.colab import files

from IPython.display import clear_output
clear_output() #for clarity

import warnings
warnings.filterwarnings("ignore")

In [8]:
df = pd.read_csv('/content/drive/MyDrive/data/corona_virus.csv')
data = []
for i in range(0,df.shape[0]):
  data.append(df['Title'][i] +'. '+ df['Abstract'][i])

In [9]:
df.head()

,Unnamed: 0,PMID,Title,Authors,Abstract
0,0,32275259,"SARS-CoV-2, SARS-CoV, and MERS-COV: A comparat...","Rabaan AA, Al-Ahmed SH, Haque S, Sah R, Tiwari...",The recent outbreak of SARS-CoV-2 that started...
1,1,19198616,The spike protein of SARS-CoV--a target for va...,"Du L, He Y, Zhou Y, Liu S, Zheng BJ, Jiang S.",Severe acute respiratory syndrome (SARS) is a ...
2,2,32347053,"[Diagnosis, treatment, control and prevention ...","Ye ZW, Jin DY.",The ongoing outbreak of the coronavirus diseas...
3,3,32333222,"The genetic sequence, origin, and diagnosis of...","Wang H, Li X, Li T, Zhang S, Wang L, Wu X, Liu J.","Coronavirus disease 2019 (COVID-19), caused by..."
4,4,32302058,"Laboratory testing of SARS-CoV, MERS-CoV, and ...","Yan Y, Chang L, Wang L.",Emerging and reemerging infectious diseases ar...


## Pre-Processing

### Stop Words Removal using apacy and scispacy

In [10]:
def preprocess(text):

  text = remove_stopwords(text) 
  text = stem_text(text)
  text = strip_multiple_whitespaces(text)
  text =  strip_punctuation(text)
  tokens = tokenize(text)
  return(list(tokens))

In [11]:
text = df['Abstract'][1]

In [15]:
nlp = spacy.load("en_core_web_sm",disable=["tagger", "parser","ner"])
def preprocess(text):
  """
  This Function will take one input argument text.
  spllit the sentences in text.
  remove stop words
  remove punctuations
  """
  stop_words = set(stopwords.words('english'))
  stemmer = PorterStemmer()
  #text = sent_tokenize(text)

  text = re.sub(r"\p{P}", lambda m: "-" if m.group(0) == "-" else "", text) # removing punctuations
  tokens = word_tokenize(text) # tokenize
  
  #stems =[stemmer.stem(word) for word in tokens] #lemmatization
  
  filtered_text = []
  for w in tokens: 
    if w not in stop_words: 
      filtered_text.append(w) 

  return(filtered_text)

In [18]:
preprocess(data[1])

['The',
 'spike',
 'protein',
 'SARS-CoV',
 '--',
 'target',
 'vaccine',
 'therapeutic',
 'development',
 'Severe',
 'acute',
 'respiratory',
 'syndrome',
 'SARS',
 'newly',
 'emerging',
 'infectious',
 'disease',
 'caused',
 'novel',
 'coronavirus',
 'SARS-coronavirus',
 'SARS-CoV',
 'The',
 'SARS-CoV',
 'spike',
 'S',
 'protein',
 'composed',
 'two',
 'subunits',
 'S1',
 'subunit',
 'contains',
 'receptor-binding',
 'domain',
 'engages',
 'host',
 'cell',
 'receptor',
 'angiotensin-converting',
 'enzyme',
 '2',
 'S2',
 'subunit',
 'mediates',
 'fusion',
 'viral',
 'host',
 'cell',
 'membranes',
 'The',
 'S',
 'protein',
 'plays',
 'key',
 'parts',
 'induction',
 'neutralizing-antibody',
 'T-cell',
 'responses',
 'well',
 'protective',
 'immunity',
 'infection',
 'SARS-CoV',
 'In',
 'Review',
 'highlight',
 'recent',
 'advances',
 'development',
 'vaccines',
 'therapeutics',
 'based',
 'S',
 'protein']

## Entities Extraction and Linking Using scispacy

In [38]:
nlp = en_core_sci_lg.load()
ner_model = en_ner_bionlp13cg_md

In [35]:
doc = nlp(text)

In [39]:
def display_entities(model,document):
    """ A function that returns a tuple of displacy image of named or unnamed word entities and
        a set of unique entities recognized based on scispacy model in use
        Args: 
            model: A pretrained model from spaCy or ScispaCy
            document: text data to be analysed"""
    nlp = model.load()
    doc = nlp(document)
    displacy_image = displacy.render(doc, jupyter=True,style='ent')
    entity_and_label = set([(X.text, X.label_) for X in doc.ents])
    return  displacy_image, entity_and_label

In [40]:
entities = display_entities(ner_model,data[5])